In [3]:
import random
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing
from sklearn.metrics import f1_score
from sklearn import  metrics
from sklearn.metrics import accuracy_score

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

np.random.seed(5)

import warnings
warnings.filterwarnings(action='ignore')

In [4]:
from konlpy.tag import Okt
okt =Okt()
text = '정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지..'
print(okt.morphs(text))

['정부', '가', '고유', '가', '대응', '을', '위해', '7월', '부터', '연말', '까지', '유류', '세', '인하', '폭', '을', '30%', '에서', '37%', '까지', '..']


In [5]:
df = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')

In [6]:
df = pd.DataFrame(df)
df

,ID,문장,유형,극성,시제,확실성,label
0,TRAIN_00000,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
1,TRAIN_00001,이어 ＂앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정＂이라며 ＂그 이전이...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
2,TRAIN_00002,정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지...,사실형,긍정,미래,확실,사실형-긍정-미래-확실
3,TRAIN_00003,"서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만, 하루 만에 ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
4,TRAIN_00004,익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실
...,...,...,...,...,...,...,...
16536,TRAIN_16536,"＇신동덤＇은 ＇신비한 동물사전＇과 ＇해리 포터＇ 시리즈를 잇는 마법 어드벤처물로, ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
16537,TRAIN_16537,"수족냉증은 어릴 때부터 심했으며 관절은 어디 한 곳이 아니고 목, 어깨, 팔꿈치, ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실
16538,TRAIN_16538,김금희 소설가는 ＂계약서 조정이 그리 어려운가 작가를 격려한다면서 그런 문구 하나 ...,사실형,긍정,과거,확실,사실형-긍정-과거-확실
16539,TRAIN_16539,1만명이 넘는 방문자수를 기록한 이번 전시회는 총 77개 작품을 넥슨 사옥을 그대로...,사실형,긍정,과거,불확실,사실형-긍정-과거-불확실


In [7]:
df['유형num'] =df['유형'].map({'사실형':0, '추론형':1, '대화형':2 , '예측형': 3})
df['극성num'] =df['극성'].map({'긍정':0, '부정':1, '미정':2})
df['시제num'] =df['시제'].map({'과거':0, '현재':1, '미래':2})
df['확실성num'] =df['확실성'].map({'확실':0, '불확실':1})

df



,ID,문장,유형,극성,시제,확실성,label,유형num,극성num,시제num,확실성num
0,TRAIN_00000,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실,0,0,1,0
1,TRAIN_00001,이어 ＂앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정＂이라며 ＂그 이전이...,사실형,긍정,과거,확실,사실형-긍정-과거-확실,0,0,0,0
2,TRAIN_00002,정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지...,사실형,긍정,미래,확실,사실형-긍정-미래-확실,0,0,2,0
3,TRAIN_00003,"서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만, 하루 만에 ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실,0,0,0,0
4,TRAIN_00004,익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
16536,TRAIN_16536,"＇신동덤＇은 ＇신비한 동물사전＇과 ＇해리 포터＇ 시리즈를 잇는 마법 어드벤처물로, ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실,0,0,0,0
16537,TRAIN_16537,"수족냉증은 어릴 때부터 심했으며 관절은 어디 한 곳이 아니고 목, 어깨, 팔꿈치, ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실,0,0,0,0
16538,TRAIN_16538,김금희 소설가는 ＂계약서 조정이 그리 어려운가 작가를 격려한다면서 그런 문구 하나 ...,사실형,긍정,과거,확실,사실형-긍정-과거-확실,0,0,0,0
16539,TRAIN_16539,1만명이 넘는 방문자수를 기록한 이번 전시회는 총 77개 작품을 넥슨 사옥을 그대로...,사실형,긍정,과거,불확실,사실형-긍정-과거-불확실,0,0,0,1


In [12]:

df['okt'] = df['문장'].apply(okt.nouns)



In [13]:
df['okt2'] = df['문장'].apply(okt.morphs)

In [23]:
df = pd.DataFrame(df)
df


,ID,문장,유형,극성,시제,확실성,label,유형num,극성num,시제num,확실성num,okt,okt2
0,TRAIN_00000,0.75%포인트 금리 인상은 1994년 이후 28년 만에 처음이다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실,0,0,1,0,"[포인트, 금리, 인상, 이후, 처음]","[0.75%, 포인트, 금리, 인상, 은, 1994년, 이후, 28년, 만에, 처음..."
1,TRAIN_00001,이어 ＂앞으로 전문가들과 함께 4주 단위로 상황을 재평가할 예정＂이라며 ＂그 이전이...,사실형,긍정,과거,확실,사실형-긍정-과거-확실,0,0,0,0,"[앞, 전문가, 주, 단위, 상황, 재, 평가, 예정, 라며, 그, 이전, 방역, ...","[이어, ＂, 앞, 으로, 전문가, 들, 과, 함께, 4, 주, 단위, 로, 상황,..."
2,TRAIN_00002,정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지...,사실형,긍정,미래,확실,사실형-긍정-미래-확실,0,0,2,0,"[정부, 고유, 대응, 위해, 연말, 유류, 세, 인하, 폭, 확대]","[정부, 가, 고유, 가, 대응, 을, 위해, 7월, 부터, 연말, 까지, 유류, ..."
3,TRAIN_00003,"서울시는 올해 3월 즉시 견인 유예시간 60분을 제공하겠다고 밝혔지만, 하루 만에 ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실,0,0,0,0,"[서울시, 올해, 즉시, 견인, 유예, 시간, 제공, 하루, 차도, 자전거, 도로,...","[서울시, 는, 올해, 3월, 즉시, 견인, 유예, 시간, 60분, 을, 제공, 하..."
4,TRAIN_00004,익사한 자는 사다리에 태워 거꾸로 놓고 소금으로 코를 막아 가득 채운다.,사실형,긍정,현재,확실,사실형-긍정-현재-확실,0,0,1,0,"[익사, 사다리, 거꾸로, 소금, 코, 가득]","[익사, 한, 자는, 사다리, 에, 태워, 거꾸로, 놓고, 소금, 으로, 코, 를,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16536,TRAIN_16536,"＇신동덤＇은 ＇신비한 동물사전＇과 ＇해리 포터＇ 시리즈를 잇는 마법 어드벤처물로, ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실,0,0,0,0,"[신동, 덤, 은, 동물, 사전, 과, 해리, 포터, 시리즈, 마법, 어드벤처, 물...","[＇, 신동, 덤, ＇, 은, ＇, 신비한, 동물, 사전, ＇, 과, ＇, 해리, ..."
16537,TRAIN_16537,"수족냉증은 어릴 때부터 심했으며 관절은 어디 한 곳이 아니고 목, 어깨, 팔꿈치, ...",사실형,긍정,과거,확실,사실형-긍정-과거-확실,0,0,0,0,"[수족, 냉증, 때, 관절, 어디, 곳, 목, 어깨, 팔꿈치, 등, 허리, 관절, ...","[수족, 냉증, 은, 어릴, 때, 부터, 심했으며, 관절, 은, 어디, 한, 곳, ..."
16538,TRAIN_16538,김금희 소설가는 ＂계약서 조정이 그리 어려운가 작가를 격려한다면서 그런 문구 하나 ...,사실형,긍정,과거,확실,사실형-긍정-과거-확실,0,0,0,0,"[김금희, 소설가, 계약서, 조정, 작가, 격려, 문구, 하나, 치기, 작가, 노고...","[김금희, 소설가, 는, ＂, 계약서, 조정, 이, 그리, 어려운가, 작가, 를, ..."
16539,TRAIN_16539,1만명이 넘는 방문자수를 기록한 이번 전시회는 총 77개 작품을 넥슨 사옥을 그대로...,사실형,긍정,과거,불확실,사실형-긍정-과거-불확실,0,0,0,1,"[방문, 자수, 기록, 이번, 전시회, 총, 개, 작품, 넥슨, 사옥, 그대로, 구...","[1만, 명이, 넘는, 방문, 자수, 를, 기록, 한, 이번, 전시회, 는, 총, ..."


In [28]:
df_x = df['문장']
df_y1 = df['유형num']
df_y2 = df['극성num']
df_y3 = df['시제num']
df_y4 = df['확실성num']

In [29]:
cv = CountVectorizer()
x_traincv = cv.fit_transform(df_x)

In [32]:
encoded_input = x_traincv.toarray()
encoded_input

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [33]:
cv.inverse_transform(encoded_input[[0]])

[array(['1994년', '28년', '75', '금리', '만에', '이후', '인상은', '처음이다', '포인트'],
       dtype='<U34')]

In [10]:
cv.get_feature_names()

['00',
 '000',
 '000cc급',
 '000cc와',
 '000ft',
 '000kg',
 '000kg인',
 '000m',
 '000m의',
 '000개가',
 '000냥과',
 '000달러',
 '000대',
 '000대가',
 '000대를',
 '000만',
 '000만원',
 '000만원대까지',
 '000명',
 '000명에',
 '000명의',
 '000번이',
 '000병씩',
 '000억',
 '000에서',
 '000여',
 '000원',
 '000장을',
 '000피트',
 '000호가',
 '006220',
 '007',
 '007스펙터',
 '01',
 '01대1',
 '02',
 '025m',
 '02점',
 '03',
 '04',
 '040',
 '05',
 '05학번으로',
 '06',
 '063',
 '06으로',
 '06점',
 '07',
 '075',
 '08',
 '09',
 '09점',
 '0과',
 '0대1로',
 '0도',
 '0명의',
 '0버전',
 '0세의',
 '0시',
 '0시에',
 '0신석',
 '0에',
 '0에서',
 '0원',
 '0원에서',
 '0은',
 '0의',
 '0점으로',
 '0ｍ',
 '0ｍ로',
 '10',
 '100',
 '1000',
 '1000m²까지',
 '1000가구',
 '1000개',
 '1000개를',
 '1000개와',
 '1000년',
 '1000달러가',
 '1000대',
 '1000만',
 '1000만개',
 '1000만건씩',
 '1000만덴',
 '1000만명',
 '1000만명에',
 '1000만원',
 '1000만원을',
 '1000만원의',
 '1000만위안',
 '1000만장에',
 '1000명',
 '1000명당',
 '1000명을',
 '1000억',
 '1000억원',
 '1000억원대',
 '1000여',
 '1000여개',
 '1000여건의',
 '1000여명이',
 '1000원',
 '1000원당',
 '1000점',
 '1000조분의

In [1]:
from konlpy.tag import Okt
okt =Okt()
text = '정부가 고유가 대응을 위해 7월부터 연말까지 유류세 인하 폭을 30%에서 37%까지..'
print(okt.morphs(text))

['정부', '가', '고유', '가', '대응', '을', '위해', '7월', '부터', '연말', '까지', '유류', '세', '인하', '폭', '을', '30%', '에서', '37%', '까지', '..']


In [40]:
mnb = MultinomialNB()
y_train = df_y1.astype('int')
mnb.fit(x_traincv, y_train)


test_x = df_x
test_y = df_y1, df_y2,df_y3,df_y4

In [41]:
x_testcv = cv.transform(test_x)
predictions = mnb.predict(x_testcv)

In [42]:
accuracy_score(test_y, predictions)

ValueError: Found input variables with inconsistent numbers of samples: [4, 16541]